# Föreläsning 5

Idag kommer vi titta på skattningar och urval.

#### Disposition
* Urval
    * Vikten av slumpmässiga urval
    * Svarsbortfall
    * Urvalsbias
* Intervallskattning
    * Konfidensintervall
    * Signifikansnivå

    https://aegis4048.github.io/comprehensive_confidence_intervals_for_python_developers

# Urval
När vi vill göra undersökningar eller mätningar av något så är det oftast in görbart att undersöka en hel population. Därför måste vi göra ett urval av populationen. Hur vi gör detta är viktigt för att få en bra skattning av populationen.

## Vikten av slumpmässiga urval (urvalsbias)
Det är viktigt att vi gör slumpmässiga urval. Om vi inte gör det så kan vi få ett urval som inte är representativt för populationen. Detta kan leda till att vi får felaktiga slutsatser om populationen.


### Exempel
Vi vill undersöka hur många som använder sig av olika försäkringar. Vi ringer upp 100 personer och frågar dem om de har en försäkring. Vi får följande svar:

|Försäkring|Antal|
|:---:|:---:|
|Ingen|40|
|Hemförsäkring|25|
|Fordon|10|
|Hemforsäkring och fordon|15|

Vi kan se att vi fått ett urval som inte är representativt för populationen. Det är väldigt få som har en hemförsäkring och fordon. Om vi hade gjort ett slumpmässigt urval så hade vi fått ett urval som var mer representativt för populationen.


## Svarsbortfall
När vi gör en undersökning så kan det hända att vissa personer inte svarar. Detta kallas svarsbortfall. Precis som för urvalsbias så kan detta leda till att vi får felaktiga slutsatser om populationen.


## Urvalsbias
När vi gör ett urval så kan vi få ett urval som inte är representativt för populationen. Detta kallas urvalsbias. Om vi inte gör ett slumpmässigt urval så kan vi få ett urval som inte är representativt för populationen. Detta kan leda till att



#### Simulering av urvalsbias !!!OBS!!! Inte klart

För att få en känsla för hur stor urvalsbias kan vara så kan vi simulera ett urval och utvärdera vårt experiment.

Vi börjar med att definiera hyperparametrarna för simuleringen.

In [ ]:
# !pip install tqdm
# !pip install hvplot

from numpy import mean, std
import numpy as np
from math import sqrt
import numpy.random as npr
import scipy.stats as stats
from tqdm import tqdm
import hvplot.pandas
import pandas as pd

In [ ]:
N = 1000 	# number of features 
n = 20   	# number of samples to draw to each feature
a = 0.05	# significance level
μ_required = 1.  # improvement threshold
Z = abs(stats.t.ppf(a, n-1)) # Z score from t-test 

Vi definerar en normalfördelad slumpvariabel ${Xᵢ}$, vars medelvärde $\mu$ och standardavvikelse $\sigma$ genereras från en annan fördelning.

In [ ]:
def make_a_feature():
    μμ = 0.5
    σμ = 0.5
    μ = npr.normal(μμ, σμ)
    σ = npr.exponential(scale=σμ)
    return μ, σ   

Sedan samplar vi N features, gör t-test och kontrollerar Bias > 0 ​tillstånd enligt ovan och samlar in bias_featuresoch unbias_features.

In [ ]:
bias_features = []
bias_estimates = []
unbias_features = []
unbias_estimates = []

for _ in tqdm(range(N)):
    μ, σ = make_a_feature()
    samples = [npr.normal(μ, σ) for _ in range(n)]
    s = std(samples)
    m = mean(samples)
    reject_h0 = (sqrt(n) * (m - μ_required) / s) > Z  # could use this feature
    if reject_h0:
        if μ < s * Z / sqrt(n) + μ_required:
            bias_features.append((μ, σ))
            bias_estimates.append((m, s))
        else:
            unbias_features.append((μ, σ))
            unbias_estimates.append((m, s))

100%|██████████| 1000/1000 [00:00<00:00, 14091.16it/s]


In [ ]:
num_unbias_plot_data = []
for n in tqdm(range(20, 601, 30)):
    bias = 0
    unbias = 0
    for _ in range(N):
        μ, σ = make_a_feature()
        samples = [npr.normal(μ, σ) for _ in range(n)]
        s = std(samples)
        m = mean(samples)
        reject_h0 = (sqrt(n) * (m - μ_required) / s) > Z  # could use this feature
        if reject_h0:
            if μ < s * Z / sqrt(n) + μ_required:
                bias += 1
            else:
                unbias += 1
    num_unbias_plot_data.append((n, bias, unbias))

100%|██████████| 20/20 [00:06<00:00,  2.94it/s]


In [ ]:
df = pd.DataFrame(num_unbias_plot_data, columns=['Sample Num', 'Num of Bias Feat', 'other'])
df

,Sample Num,Num of Bias Feat,other
0,20,13,89
1,50,9,98
2,80,5,120
3,110,8,115
4,140,4,112
5,170,9,114
6,200,6,115
7,230,2,135
8,260,4,122
9,290,5,147


In [ ]:
df.hvplot.line(x='Sample Num', y='Num of Bias Feat', title='More Samples → Less Bias')

:Curve   [Sample Num]   (Num of Bias Feat)

In [ ]:
df = pd.DataFrame(list(zip(abs(np.array(unbias_estimates)[:, 0] - np.array(unbias_features)[:, 0]),
                   abs(np.array(bias_estimates)[:, 0] - np.array(bias_features)[:, 0]))), columns=['Est Err of Unbias Feat', 'Est Err of Bias Feat'])
                   
df

,Est Err of Unbias Feat,Est Err of Bias Feat
0,0.018305,0.053283
1,0.004885,0.228530
2,0.169870,0.333658
3,0.013642,0.211135
4,0.010498,0.444959
5,0.020296,0.049944
6,0.070034,0.064483
7,0.072487,0.017657
8,0.043657,0.439319
9,0.318930,0.164767


In [ ]:
df.hvplot.density(y=['Est Err of Unbias Feat', 'Est Err of Bias Feat'], title='Histogram of Esimation Error')

:NdOverlay   [Variable]
   :Distribution   [value]   (Density)

In [ ]:
n = 200000
bias = 0
unbias = 0
for _ in tqdm(range(N)):
    μ, σ = make_a_feature()
    samples = [npr.normal(μ, σ) for _ in range(n)]
    s = std(samples)
    m = mean(samples)
    reject_h0 = (sqrt(n) * (m - μ_required) / s) > Z  # could use this feature
    if reject_h0:
        if μ < s * Z / sqrt(n) + μ_required:
            bias += 1
        else:
            unbias += 1

100%|██████████| 1000/1000 [03:30<00:00,  4.76it/s]


In [ ]:
bias  # Enough Sampling => No Bias!

0

In [24]:
df2 = pd.DataFrame(list(zip(abs(np.array(unbias_estimates)[:, 0] - np.array(unbias_features)[:, 0]),
                   abs(np.array(bias_estimates)[:, 0] - np.array(bias_features)[:, 0]))), columns=['Est Err of Unbias Feat', 'Est Err of Bias Feat'])
                   
df2

,Est Err of Unbias Feat,Est Err of Bias Feat
0,0.018305,0.053283
1,0.004885,0.228530
2,0.169870,0.333658
3,0.013642,0.211135
4,0.010498,0.444959
5,0.020296,0.049944
6,0.070034,0.064483
7,0.072487,0.017657
8,0.043657,0.439319
9,0.318930,0.164767


In [ ]:
# Generera en normalfördelad variabel 
# Beräkna sedan konfidensintervallet för medelvärdet med olika konfidensgrader

import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# Konfidensgrader
conf = [0.95, 0.99, 0.999]

# Normalfördelad variabel med medelvärde 0 och standardavvikelse 1
mean = 0
std = .1
X = stats.norm(mean, std)

# Beräkna konfidensintervallet för medelvärdet
n = 100        # Antal observationer

x = X.rvs(n)    # Generera n observationer


In [ ]:
# Beräkna konfidensintervallet för medelvärdet
xbar = np.mean(x)   # Beräkna medelvärdet
s = np.std(x, ddof=1)   # Beräkna standardavvikelsen

# Gränsvärden för konfidensintervallet vid olika konfidensgrader
lower = []
upper = []
for c in conf:
    lower.append(xbar - stats.norm.ppf((1+c)/2)*s/np.sqrt(n))
    upper.append(xbar + stats.norm.ppf((1+c)/2)*s/np.sqrt(n))
    print('Konfidensgrad: ', c)
    print('Konfidensintervallet: [', lower[-1], ', ', upper[-1], ']\n')

# Plotta histogrammet för de genererade observationerna tillsammans med konfidensintervallen för medelvärdet
height, bins, patches = plt.hist(x, bins=20, density=True, alpha=0.5)

plt.fill_betweenx([0, height.max()/2], lower[0], upper[0], color='r', alpha=0.1)
# plt.fill_betweenx([0, 1], lower[0], upper[0], color='r', alpha=0.1)
plt.fill_betweenx([0, height.max()], lower[1], upper[1], color='b', alpha=0.1)
plt.fill_betweenx([0, height.max()*1.5], lower[2], upper[2], color='k', alpha=0.1)
plt.legend(['Konfidensgrad 95%', 'Konfidensgrad 99%', 'Konfidensgrad 99.9%'])
plt.title('Konfidensintervall för medelvärdet')
plt.show()